In [ ]:
from datetime import datetime, timedelta
import os
import openpyxl
import os
import pandas as pd

ROOT_DIR = './'

MARKS = ['СКС', "СКД", "ДСТ", "СБС","ДССК"]
print("Ready!")

In [ ]:
#Загрузка образца отчёта
wb = openpyxl.load_workbook('weekly_basis.xlsx')
report_sheet = wb['Форма горизонталь1']


report_titles = report_sheet['A1':'AL3']
hiden_data = report_sheet['A4':'G8']
rows = {'sks':report_sheet['G4':'AL4'],'skd':report_sheet['G5':'AL5'],'tep35':report_sheet['G6':'AL6'],
        'tep50':report_sheet['G7':'AL7'],'total':report_sheet['G8':'AL8']}




In [ ]:
#test zone
wb_test = openpyxl.load_workbook('test_formulas.xlsx')
test_sheet = wb_test['test']
create_formulas_cols(test_sheet)
find_total(test_sheet)
copy_cells(rows['total'], test_sheet)

In [ ]:
#Подгрузка отчётов из SAP 

#добавить проверку на верность даты начала и конца выгрузки
prod_report  = pd.read_excel(os.path.join(ROOT_DIR, 'ZPP_PROD_REPORT.XLSX'),sheet_name = 'Sheet1', thousands = ',').drop(columns='Продукт').rename(columns={'Продукт.1':'Продукт'})
mb52_report  = pd.read_excel(os.path.join(ROOT_DIR, 'MB52.XLSX'),sheet_name = 'Sheet1', thousands = ',')



In [ ]:
orion = pd.read_excel(os.path.join(ROOT_DIR, 'ORION.XLSX'),sheet_name = 'Last_ОАО Воронежсинтезкаучук', thousands = ',')
        
orion

In [ ]:

prepare_orion_fact(orion)

In [ ]:

orion_fact2
#pd.concat([orion_fact1,orion_fact2])

In [ ]:
#pd.concat([orion_fact1,orion_fact2])


In [ ]:
#Используемые функции(отлаженные и протестированные)
#печатает слайс
def print_row(row_slice):
    for cell_obj in row_slice:
        for cell in cell_obj:
            print(cell.value, cell.column, cell.coordinate)
#создает эксель, в который пишутся результаты
def create_resulting_wb():
    wb_result = openpyxl.Workbook()
    wb_result.create_sheet(title = 'Отчёт', index = 0)
    result_sheet = wb_result['Отчёт']
    for irow in range(8):
        result_sheet.append([])
    wb_result.save('weekly_report.xlsx')
    return wb_result
#копирую инфу из одного экселя в другой
def copy_cells(slice_to_copy,sheet_we_copy_in):
    for cells_obj in slice_to_copy:
        for cell in cells_obj:
            cell_tmp = sheet_we_copy_in.cell(row = cell.row, column = cell.column)
            cell_tmp.value = cell.value
    wb_result.save('weekly_report.xlsx')
    #wb_test.save('weekly_report.xlsx')
#Посчитать последнюю строку    
#Вызывать последней
def find_total(rep_sheet):
    total_positions = []
    for cell_obj in rows['total']:
        for cell in cell_obj:
            if type(cell.value) == str:
                total_positions.append((cell.value, cell.column, cell.coordinate))
    for i in total_positions:
        try:
            rep_sheet[i[2]] = (
                                  float(rep_sheet.cell(column=i[1],row=4).value) + 
                                  float(rep_sheet.cell(column=i[1],row=5).value) + 
                                  float(rep_sheet.cell(column=i[1],row=6).value) + 
                                  float(rep_sheet.cell(column=i[1],row=7).value)
                                  )
        except ValueError:
            #print(i[2])
            rep_sheet[i[2]] = ''
    #return rep_sheet['G8':'AL8']

def put_info_in_col(col_n, data_dict, sheet_we_put_in):
    cell_tmp = sheet_we_put_in.cell(row = 4, column = col_n)#CKC
    cell_tmp.value =data_dict['СКС']
    cell_tmp = sheet_we_put_in.cell(row = 5, column = col_n)#CKД+ДССК+СКД-НД
    cell_tmp.value =data_dict['СКД']
    cell_tmp = sheet_we_put_in.cell(row = 6, column = col_n)#ДСТ
    cell_tmp.value =data_dict['ДСТ']
    cell_tmp = sheet_we_put_in.cell(row = 7, column = col_n)#CБС
    cell_tmp.value =data_dict['СБС']
    wb_result.save('weekly_report.xlsx')


#Получаем данные для 17стобца из ZPP_PROD_REPORT -> вариант VSK_PERIOD_PRD (Отчёт ВСК производства и сортировки продукции. Паспортизация)
#вызывать предпоследней
def get_column_17_info(df_zpp_prod_report):
    #['СКС', "СКД"+"ДССК", "ДСТ", "СБС"]
    RESULT = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    for i in range(prod_report.shape[0]):
        row_tmp = prod_report.iloc[i,:]
        row_tmp['Продукт'] = str(row_tmp['Продукт'])
        if MARKS[0] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#CKC - верно
            RESULT['СКС'] += row_tmp['Паспортизация'].astype(float) 
            continue
        if MARKS[1] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#СКД - верно
            RESULT['СКД'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[4] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#ДССК - верно
            RESULT['СКД'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[2] in row_tmp['Продукт']:# and not 'ЭП' in row_tmp['Продукт']:#ДСТ
            RESULT['ДСТ'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[3] in row_tmp['Продукт']:# and not 'ЭП' in row_tmp['Продукт']:#СБС
            RESULT['СБС'] += row_tmp['Паспортизация'].astype(float) 
            continue
    return RESULT

#Получаем данные для 10стобца из MB52 -> вариант BOGDANOVA
def get_column10_info(df_mb52_report):
    RESULT_CLEAR = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    RESULT_BRAK611 = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    RESULT_EP = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0,'ЭП-ТЭП':0}

   #готовый продукт без ЭП и брака 611
    for i in MARKS:
        #1 без брака и ЭП
        if i == 'ДССК':
            RESULT_CLEAR['СКД'] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == False)]['СвобИспользЗапас'].sum()#.reset_index().drop(columns = 'index')
        else:
            if(i == 'СБС') or (i =='ДСТ'):
                RESULT_CLEAR[i] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True)  & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == False)]['СвобИспользЗапас'].sum()#.reset_index().drop(columns = 'index')
            else:
                RESULT_CLEAR[i] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == False)]['СвобИспользЗапас'].sum()
       #2 БРАК611
        if i == 'ДССК':
            RESULT_BRAK611['СКД'] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == True)]['СвобИспользЗапас'].sum()#.reset_index().drop(columns = 'index')
        else:
            if(i == 'СБС') or (i =='ДСТ'):
                RESULT_BRAK611[i] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True)  & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == True)]['СвобИспользЗапас'].sum()#.reset_index().drop(columns = 'index')
            else:
                RESULT_BRAK611[i] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == True)]['СвобИспользЗапас'].sum()
        #3 ЭП№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№№
    RESULT_EP['СКД'] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-ДССК') == True)]['СвобИспользЗапас'].sum()#.reset_index().drop(columns = 'index')
        #Как отличить ЭП ДСТ от ЭП СБС?
    RESULT_EP['ЭП-ТЭП'] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-ТЭП') == True)]['СвобИспользЗапас'].sum()#.reset_index().drop(columns = 'index')
    RESULT_EP['СКС'] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-СКС') == True)]['СвобИспользЗапас'].sum()
    RESULT_EP['СКД'] += df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-ПБ') == True)]['СвобИспользЗапас'].sum()

    #return RESULT_CLEAR, RESULT_BRAK611, RESULT_EP
    return 'Продукт:'+str(RESULT_CLEAR), 'БРАК:'+ str(RESULT_BRAK611), 'ЭП:'+str(RESULT_EP)

#Считаем все столбцы под формулами
#11,12,18,19,20,21,24,,27,30,33,34
def create_formulas_cols(resulting_sheet):
    #[[куда поместить результат, оператор1, действие, оператор2, оператор3]-  действие одно на все операторы
    formulas= [[11, 10, '/',8], [12, 13, '+',14, 15], [18, 17, '/',16], [19, 28, '+',31, 25], [20, 26, '+', 29, 32], 
              [21, 20, '/',19], [24, 23, '/',22],[27, 26, '/',25], [30, 29, '/', 28], [33, 32, '/', 31], [34, 23, '-',22]]
    for i in formulas:
        print(i)
        
            
        if i[2] == '/':
            try:
                resulting_sheet.cell(column=i[0],row=4).value = float(resulting_sheet.cell(column=i[1],row=4).value)/float(resulting_sheet.cell(column=i[3],row=4).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=4).value = '-'
            try:
                resulting_sheet.cell(column=i[0],row=5).value = float(resulting_sheet.cell(column=i[1],row=5).value)/float(resulting_sheet.cell(column=i[3],row=5).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=5).value = '-'
            try:
                resulting_sheet.cell(column=i[0],row=6).value = float(resulting_sheet.cell(column=i[1],row=6).value)/float(resulting_sheet.cell(column=i[3],row=6).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=6).value = '-'
            try:
                resulting_sheet.cell(column=i[0],row=7).value = float(resulting_sheet.cell(column=i[1],row=7).value)/float(resulting_sheet.cell(column=i[3],row=7).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=7).value = '-'

        if i[2] == '+':
            resulting_sheet.cell(column=i[0],row=4).value = float(resulting_sheet.cell(column=i[1],row=4).value)+float(resulting_sheet.cell(column=i[3],row=4).value)+(resulting_sheet.cell(column=i[4],row=4).value)
            resulting_sheet.cell(column=i[0],row=5).value = float(resulting_sheet.cell(column=i[1],row=5).value)+float(resulting_sheet.cell(column=i[3],row=5).value)+(resulting_sheet.cell(column=i[4],row=5).value)
            resulting_sheet.cell(column=i[0],row=6).value = float(resulting_sheet.cell(column=i[1],row=6).value)+float(resulting_sheet.cell(column=i[3],row=6).value)+(resulting_sheet.cell(column=i[4],row=6).value)
            resulting_sheet.cell(column=i[0],row=7).value = float(resulting_sheet.cell(column=i[1],row=7).value)+float(resulting_sheet.cell(column=i[3],row=7).value)+(resulting_sheet.cell(column=i[4],row=7).value)

        if i[2] == '-':
            resulting_sheet.cell(column=i[0],row=4).value = float(resulting_sheet.cell(column=i[1],row=4).value)-float(resulting_sheet.cell(column=i[3],row=4).value)
            resulting_sheet.cell(column=i[0],row=5).value = float(resulting_sheet.cell(column=i[1],row=5).value)-float(resulting_sheet.cell(column=i[3],row=5).value)
            resulting_sheet.cell(column=i[0],row=6).value = float(resulting_sheet.cell(column=i[1],row=6).value)-float(resulting_sheet.cell(column=i[3],row=6).value)
            resulting_sheet.cell(column=i[0],row=7).value = float(resulting_sheet.cell(column=i[1],row=7).value)-float(resulting_sheet.cell(column=i[3],row=7).value)
    wb_result.save('weekly_report.xlsx')
    
    
    #wb_test.save('weekly_report.xlsx')
    #Возвращает датасет с данным ППР
    #Тип объекта РО - запланировано по ППР
    #Тип объекта ОТКЛОНЁН - ФАКТ, но необходимо смотреть на таблицу со сдвигом +102
    #TODO добавить фильтр по РО
    def prepare_orion_PPR(orion_df):
    orion_fact1 = orion.iloc[5:,1:4].rename(columns = {'Unnamed: 1':'Наименование позиции','Unnamed: 2':'Тип объекта','Unnamed: 3':'Метод отгрузки'}).reset_index().drop(columns = 'index')
    orion_fact2 = orion.iloc[1:,19:50]
    v = orion_fact2.iloc[0,:].to_list()
    k = orion_fact2.columns.to_list()
    cols_new = {}
    for i in range(len(k)):
        cols_new.update({k[i]:v[i]})
    orion_fact2 = orion_fact2.rename(columns = cols_new).drop([1,2,3,4]).reset_index().drop(columns = 'index')
    return orion_fact1.merge(orion_fact2, right_index = True, left_index = True)

In [ ]:
#Основной список вызов функций (main)
wb_result = create_resulting_wb()
result_sheet = wb_result['Отчёт']
copy_cells(report_titles, result_sheet)
copy_cells(hiden_data, result_sheet)
#17 столбец
put_info_in_col(17,get_column_17_info(prod_report),result_sheet)
#предпоследняя
create_formulas_cols(result_sheet)
#последняя строка
find_total()
copy_cells(rows['total'], result_sheet)

In [ ]:
#MARKS = ['СКС', "СКД", "ДСТ", "СБС","ДССК"]


In [ ]:
mb52_report[(mb52_report['Краткий текст материала'].str.contains(i) == True) & (mb52_report['Краткий текст материала'].str.contains('ЭП') == False) & (mb52_report['Краткий текст материала'].str.contains(' 611') == False)]

In [ ]:
RESULT = [0,0]
for i in range(prod_report.shape[0]):
    row_tmp = prod_report.iloc[i,:]
    row_tmp['Продукт'] = str(row_tmp['Продукт'])
    if MARKS[1] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#СКД
        RESULT[1] += row_tmp['Паспортизация'].astype(float)
        print(row_tmp)

In [ ]:
RESULT1 = [0,0,0]
for i in range(prod_report.shape[0]):
    row_tmp = prod_report.iloc[i,:]
    row_tmp['Продукт'] = str(row_tmp['Продукт'])
    if z in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#ДССК
        RESULT1[2] += row_tmp['Паспортизация'].astype(float)
        print(row_tmp)        
RESULT1

In [ ]:
z = prod_report.iloc[1370,0].split(' ')[1]
z